## Example- electron-electron collision operator test for l=0

This example tests relaxation to Maxwellian of the l=0 electron harmonic due to e-e collisions. 

This notebook is the v2.0.0 rewrite which corresponds to the first test in Section 5.2.2. in the ReMKiT1D code paper. 


In [ ]:
import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

import RMK_support as rmk
import RMK_support.common_models as cm
from RMK_support import node,varFromNode

### Context initialization

In [ ]:
rk = rmk.RMKContext()
rk.IOContext = rmk.IOContext(HDF5Dir="./RMKOutput/RMK_ee_coll_test/" )

### Grid setup

In [ ]:
xGrid = np.ones(1) # 0D

uniformVGrid = False

dv0 = 0.0307
cv = 1.03 #Change to 1.025 for the short grid
Nv = 120
vGrid = [dv0/2]
for i in range(1,Nv):
    vGrid.append(vGrid[i-1]*cv)
if uniformVGrid:
    maxV = sum(vGrid)
    vGrid = maxV/Nv*np.ones(Nv)
    
lMax = 0 
rk.grid = rmk.Grid(xGrid,np.array(vGrid),lMax,interpretXGridAsWidths=True,interpretVGridAsWidths=True)

### Variable initialization

In [ ]:
T0 = 0.5
n0 = 1.0
T = T0*np.ones(rk.grid.numX())
n = n0 * np.ones(rk.grid.numX())
W = 3*n*T/2

fInit = np.zeros([rk.grid.numX(),rk.grid.numH(),rk.grid.numV()])
for i in range(rk.grid.numX()):
    fInit[i,rk.grid.getH(0)-1,:] = (T0*np.pi)**(-1.5) * n0 * np.exp(-rk.grid.vGrid**2/T0) + (T0*np.pi)**(-1.5) * 0.1 *n0 * np.exp(-(rk.grid.vGrid-3)**2/T0) 

f = rmk.Variable("f",rk.grid,data=fInit,isDistribution=True)
W = rmk.Variable("W",rk.grid,derivation=rk.textbook["energyMoment"],derivationArgs=["f"])
n = rmk.Variable("n",rk.grid,derivation=rk.textbook["densityMoment"],derivationArgs=["f"])
T =  varFromNode("T",rk.grid,node=(2/3)*node(W)/node(n))

rk.variables.add(f,W,n,T)

### Adding e-e collision operator model for l = 0

The collision operator is implemented in common_models.py and only used here.

In [ ]:
rk.models.add(cm.eeCollIsotropic(f,T,n,rk.norms,rk.grid,rk.textbook))

### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
integrator = rmk.BDEIntegrator("BDE",nonlinTol=1e-12,absTol=10.0,convergenceVars=[W,f])
integrationStep = rmk.IntegrationStep("BE",integrator)
integrationStep.add(rk.models) 
rk.integrationScheme = rmk.IntegrationScheme(dt=0.05,steps=integrationStep) 
rk.integrationScheme.setFixedNumTimesteps(1200,5) 

#### Generate a LaTeX summary of the ReMKiT1D run 

In [ ]:
rk.generatePDF("Isotropic e-e collision test")

### Create config file

In [ ]:
rk.writeConfigFile()

### Data analysis


#### Loading data

In [ ]:
loadedData = rk.loadSimulation()
dataset = loadedData.dataset

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=80,dpi=150)

### Expected Maxwellian comparison

In [ ]:
f0_analytic=np.zeros((numFiles+1,rk.grid.numV()))

for i in range(numFiles+1):
    T = dataset["T"].values[i,0]
    n = dataset["n"].values[i,0]
    f0_analytic[i,:] = np.pi**(-1.5) * T ** (-1.5) * n * np.exp(-rk.grid.vGrid**2/T) 

In [ ]:
dataName = 'f'

In [ ]:
#this can take a few minutes
curveDict = {dataset.coords["t"].data[t]: hv.Curve(dataset[dataName][{"t":t,"x":0,"h":0}])*hv.Curve((rk.grid.vGrid,f0_analytic[t,:])) for t in range(numFiles+1)}
kdims = [hv.Dimension(('t', 'Time'),unit=dataset.coords["t"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims)

### Reproducing plots from the paper

In [ ]:
def fixed_aspect_ratio_loglin(plot, element):
    '''
    Set a fixed aspect ratio on matplotlib loglog plots 
    regardless of axis units
    '''
    ratio = plot._aspect_param_value
    ax = plot.handles['axis']
    xvals,yvals = ax.axes.get_xlim(),ax.axes.get_ylim()

    xrange = xvals[1]-xvals[0]
    yrange = np.log(yvals[1])-np.log(yvals[0])
    ax.set_aspect(ratio*(xrange/yrange), adjustable='box')

    ax.axes.set_xscale('function', functions=(lambda x : x**2,lambda x : x**(1/2)))
    xticks = np.linspace(xvals[0]**2,xvals[1]**2,10)
    ax.axes.set_xticks(xticks**(1/2),map(str,np.around(xticks,1)))

In [ ]:
plotLin = hv.Overlay([hv.Scatter(dataset[dataName][{"t":t,"x":0,"h":0}],label='${:.2f}t_0$'.format(dataset.coords["t"].data[t])).opts(marker=hv.Cycle(['o','x','^']),s=8.0) for t in [0,1,2,20,120,240]])* hv.Curve((rk.grid.vGrid,f0_analytic[0,:]),label='Maxwellian').opts(color='k',linestyle='--')
plotLin.opts(xlim=(0,14),logy=True,ylim=(1e-12,2),aspect=30, hooks=[fixed_aspect_ratio_loglin],xlabel="E (eV)",fontsize={'xticks':6.0,'legend':8.0},legend_cols=1,legend_position='top_right')

Rerun with different "cv" values to get the temperature errors in Figure 9b

In [ ]:
tempErrShortGrid=abs(dataset["T"][:,0]-dataset["T"][0,0])/dataset["T"][0,0]
sum(vGrid)

In [ ]:
tempErrLongGrid=abs(dataset["T"][:,0]-dataset["T"][0,0])/dataset["T"][0,0]
sum(vGrid)

In [ ]:
errorPlot = hv.Scatter(tempErrShortGrid,label="$v_{max}=11.27v_{th}$").opts(s=6.0,marker='x')*hv.Scatter(tempErrLongGrid,label="$v_{max}=17.25v_{th}$").opts(s=6.0)
errorPlot.opts(logy=True,ylim=(1e-16,1e-6),aspect=0.5,legend_cols=1,ylabel='$\delta T$')

In [ ]:
hv.output(fig='pdf')
hv.save(plotLin, 'e-eMaxwellian.pdf', dpi=144)
hv.save(errorPlot,'e-eMaxwellianTError.pdf',dpi=144)